In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time

In [2]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime

# Setup the Chrome WebDriver
chromedriver_path = ChromeDriverManager().install()
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service)

company_name = "Tesla"
site_name = "bbc"
num_days_ago = "7"

# URL to scrape
url = f'https://news.google.com/search?q={company_name}%20site%3A{site_name}.com%20when%3A{num_days_ago}d&hl=en-GB&gl=GB&ceid=GB%3Aen'
driver.get(url)

# Data lists
data = []

# Find elements by xpaths
article_elements = driver.find_elements(By.XPATH, '//a[@class="JtKRv"]')
datetime_elements = driver.find_elements(By.XPATH, '//time[@class="hvbAAd"]')

# Check if the number of articles and datetimes are the same
if len(article_elements) == len(datetime_elements):
	for article, datetime_element in zip(article_elements, datetime_elements):
		article_name = article.text

		# Check if the article title contains the company name
		if company_name.lower() in article_name.lower():
			# Extracting the datetime string and parsing it
			datetime_str = datetime_element.get_attribute('datetime')
			parsed_datetime = datetime.strptime(datetime_str, "%Y-%m-%dT%H:%M:%SZ")
			date = parsed_datetime.date()
			timestamp = parsed_datetime.time()
			
			# Extract article link
			article_link = article.get_attribute('href')
	
			# Append to data list
			data.append([date, timestamp, article_name, article_link, site_name])

# Create a DataFrame
columns = ['Date', 'Time', 'Article Name', 'Article Link', 'Site Name']
df = pd.DataFrame(data, columns=columns)

# Close the driver
driver.quit()

# Output DataFrame
df

,Date,Time,Article Name,Article Link,Site Name
0,2024-06-14,11:37:44,Is Elon Musk worth his £44bn Tesla pay package?,https://news.google.com/articles/CBMiLmh0dHBzO...,bbc
1,2024-06-15,17:51:06,Elon Musk wins record-breaking Tesla pay deal ...,https://news.google.com/articles/CBMiLmh0dHBzO...,bbc
2,2024-06-11,23:02:10,Tesla pay fight tests power of Elon Musk's mys...,https://news.google.com/articles/CBMiLmh0dHBzO...,bbc


In [3]:
# Function to initialize Selenium WebDriver
def init_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('headless')  # Run in headless mode (no GUI)
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    return driver

# Function to scrape the content of a single article
def scrape_article(url, driver):
    try:
        driver.get(url)
        time.sleep(3)  # Wait for the page to load
        text_blocks = driver.find_elements(By.XPATH, "//div[@data-component='text-block']")
        article_content = ' '.join([block.text for block in text_blocks if block.text.strip() != ''])
        return article_content
    except Exception as e:
        print(f"Error scraping {url}: {e}")
        return None

# Main function to read CSV and scrape articles
def main():
	# # Load the CSV file
	# df = pd.read_csv('News Data.csv')

	# Initialize the WebDriver
	driver = init_driver()

	# Column for storing scraped content
	df['Article Content'] = ''

	# Iterate through each row in the DataFrame
	for index, row in df.iterrows():
		print("Scrapping Started for:", index)
		content = scrape_article(row['Article Link'], driver)
		df.at[index, 'Article Content'] = content
		print("Scrapping Ended for:", index)
    
    # # Save the DataFrame with the new data
    # df.to_csv('path_to_your_file_with_content.csv', index=False)

# Quit the driver
driver.quit()

if __name__ == '__main__':
    main()

Scrapping Started for: 0
Scrapping Ended for: 0
Scrapping Started for: 1
Scrapping Ended for: 1
Scrapping Started for: 2
Scrapping Ended for: 2


In [4]:
df

,Date,Time,Article Name,Article Link,Site Name,Article Content
0,2024-06-14,11:37:44,Is Elon Musk worth his £44bn Tesla pay package?,https://news.google.com/articles/CBMiLmh0dHBzO...,bbc,"Elon Musk might, once again, get his own way.\..."
1,2024-06-15,17:51:06,Elon Musk wins record-breaking Tesla pay deal ...,https://news.google.com/articles/CBMiLmh0dHBzO...,bbc,Tesla shareholders have backed a record-breaki...
2,2024-06-11,23:02:10,Tesla pay fight tests power of Elon Musk's mys...,https://news.google.com/articles/CBMiLmh0dHBzO...,bbc,"In 2018, Tesla shareholders approved the bigge..."


In [6]:
print(df["Article Content"][0])

Elon Musk might, once again, get his own way.
His Tesla pay deal - worth up to $56bn (£44bn) depending on the firm's share price - which has now been backed by shareholders is 75% of the entire spending for schools in England in 2024-25 (£60bn) and around a quarter of the budget for the NHS (£192bn).
To his many admirers, Mr Musk is worth every single cent which comes his way - and shareholders agreed, with around 72% of the voting shares backing the deal.
His businesses include Tesla, SpaceX, X (formerly Twitter), Starlink, Neuralink and X.ai, his latest AI project.
You could argue Tesla opened up the US market for electric vehicles, SpaceX has just sent the world’s most powerful rocket into space, and a man who volunteered to be the first human to be implanted with a Neuralink microchip into his brain can now control a computer using his thoughts.
Today, the $56bn question is: would this portfolio have been as successful without him? You could say Mr Musk's pay deal was really about 